# Core10_12 — Policy Invariance & Versioned Deterministic Replay

## 목적
정책(policy) 버전이 변경되더라도,
- 어떤 정책이
- 어떤 설정으로
- 어떤 결과를 만들었는지

를 **완전히 추적 가능**하며,
동일 입력에 대해 **결정론적 재현(replay)** 이 가능함을 증명한다.

## 핵심 개념
- policy_version_id
- config hashing (content-addressed)
- deterministic replay
- replay checksum

## 산출물
- core10_12_policy_snapshot.json
- core10_12_replay_checksum_log.csv

In [1]:
from pathlib import Path
import json
import hashlib
import pandas as pd
import numpy as np
from datetime import datetime

ART_CORE10 = Path("../artifact/core10")

POLICY_PATH = ART_CORE10 / "core10_06_allocation_policy.json"
ROLLOUT_PATH = ART_CORE10 / "core10_07_operation_rollout_log.csv"

OUT_POLICY_SNAPSHOT = ART_CORE10 / "core10_12_policy_snapshot.json"
OUT_REPLAY_LOG = ART_CORE10 / "core10_12_replay_checksum_log.csv"

assert POLICY_PATH.exists(), "policy json missing"
assert ROLLOUT_PATH.exists(), "rollout log missing"

policy = json.loads(POLICY_PATH.read_text(encoding="utf-8"))
rollout = pd.read_csv(ROLLOUT_PATH)

print("loaded policy:", policy.get("policy_id"))
print("rollout rows:", len(rollout))

loaded policy: core10_06_allocation_policy_v1
rollout rows: 60


Canonical Policy Serialization

	•	key 순서 고정
	•	공백/포맷 제거
	•	해시 입력은 오직 내용만

In [2]:
def canonical_json(obj) -> str:
    """
    Deterministic JSON serialization for hashing.
    """
    return json.dumps(
        obj,
        sort_keys=True,
        separators=(",", ":"),
        ensure_ascii=False,
    )

In [3]:
policy_canonical = canonical_json(policy)

policy_hash = hashlib.sha256(
    policy_canonical.encode("utf-8")
).hexdigest()

policy_version_id = f"{policy.get('policy_id','policy')}:{policy_hash[:12]}"

policy_version_id

'core10_06_allocation_policy_v1:3c26436738ef'

In [4]:
policy_snapshot = {
    "policy_id": policy.get("policy_id"),
    "policy_version_id": policy_version_id,
    "policy_hash": policy_hash,
    "created_at": policy.get("created_at"),
    "snapshot_ts": datetime.utcnow().isoformat(),
    "policy_body": policy,
}

OUT_POLICY_SNAPSHOT.write_text(
    json.dumps(policy_snapshot, indent=2, ensure_ascii=False),
    encoding="utf-8",
)

print("✅ exported policy snapshot:")
print(OUT_POLICY_SNAPSHOT.resolve())

✅ exported policy snapshot:
/Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_12_policy_snapshot.json


In [5]:
REPLAY_CONTRACT = {
    "policy_version_id": policy_version_id,
    "seed": 42,
    "T_STEPS": rollout["step"].nunique(),
    "metrics_tracked": [
        "selected_antibody_id",
        "SoMS",
        "toggle_rate",
        "switched",
    ],
}

REPLAY_CONTRACT

{'policy_version_id': 'core10_06_allocation_policy_v1:3c26436738ef',
 'seed': 42,
 'T_STEPS': 60,
 'metrics_tracked': ['selected_antibody_id',
  'SoMS',
  'toggle_rate',
  'switched']}

In [6]:
def compute_replay_checksum(df: pd.DataFrame, cols) -> str:
    """
    Compute deterministic checksum from selected rollout columns.
    """
    payload = (
        df[cols]
        .astype(str)
        .agg("|".join, axis=1)
        .str.cat(sep="||")
    )
    return hashlib.sha256(payload.encode("utf-8")).hexdigest()

In [7]:
checksum = compute_replay_checksum(
    rollout,
    cols=REPLAY_CONTRACT["metrics_tracked"],
)

replay_log = pd.DataFrame([{
    "policy_version_id": policy_version_id,
    "policy_hash": policy_hash,
    "seed": REPLAY_CONTRACT["seed"],
    "T_STEPS": REPLAY_CONTRACT["T_STEPS"],
    "checksum": checksum,
    "computed_at": datetime.utcnow().isoformat(),
}])

replay_log

,policy_version_id,policy_hash,seed,T_STEPS,checksum,computed_at
0,core10_06_allocation_policy_v1:3c26436738ef,3c26436738efbfc2179d2bb7d8c398923ac76411d20250...,42,60,e4de3b3a2b0a557071a025298dacb093a61205bf19508e...,2026-01-10T02:46:14.462697


In [8]:
replay_log.to_csv(OUT_REPLAY_LOG, index=False)

print("✅ exported replay checksum log:")
print(OUT_REPLAY_LOG.resolve())

✅ exported replay checksum log:
/Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_12_replay_checksum_log.csv


## Governance Interpretation

- 정책은 `policy_version_id`로 **내용 기반 식별**
- rollout 결과는 checksum으로 **결정론적 지문화**
- 정책이 변경되면:
  - hash 변경
  - policy_version_id 변경
  - checksum 변경

즉,
> **“정책은 바뀔 수 있으나, 결과는 언제나 추적·재현 가능하다.”**

이는 Core10이
- 실험 시스템이 아니라
- **운영·감사 가능한 거버넌스 시스템**임을 증명한다.